In [11]:
from google.cloud import speech
from google.cloud import storage
import wave
import io

In [1]:
def frame_rate_channel(audio_file_name):
    with wave.open(audio_file_name, 'rb') as wave_file:
        frame_rate = wave_file.getframerate()
        channels = wave_file.getnchannels()
        return frame_rate,channels

def Config_GGC(sample_rate_hertz = 44100,
                audio_channel_count = 1,
                model = None,
                enable_automatic_punctuation=True):
    if model != None:
        config_wav_enhanced = speech.RecognitionConfig(
            sample_rate_hertz = sample_rate_hertz,
            enable_automatic_punctuation=enable_automatic_punctuation,
            language_code = 'vi-VN',
            audio_channel_count=audio_channel_count,
            model = model,
        )
    else :
        config_wav_enhanced = speech.RecognitionConfig(
            sample_rate_hertz = sample_rate_hertz,
            enable_automatic_punctuation=True,
            language_code = 'vi-VN',
            audio_channel_count=audio_channel_count
        )
    return config_wav_enhanced

def Config_noPunc(sample_rate_hertz = 44100,
                audio_channel_count = 1,
                model = None,
                enable_automatic_punctuation=True):
    if model is not None:
        config_wav_enhanced = speech.RecognitionConfig(
            sample_rate_hertz = sample_rate_hertz,
            language_code = 'vi-VN',
            audio_channel_count=audio_channel_count,
            model = model,
        )
    else :
        config_wav_enhanced = speech.RecognitionConfig(
            sample_rate_hertz = sample_rate_hertz,
            language_code = 'vi-VN',
            audio_channel_count=audio_channel_count
        )
    return config_wav_enhanced

def is_digit(word):
    try:
        int(word)
        return True
    except ValueError:
        pass
    return False

def ConvertDate(text):
    month=' tháng '
    year=' năm '
    for index in range(0,len(text)):
        try:
            if (text.index(month,index)==index):
                dateNum = text[index -1]
                monthNum = text[index + len(month)]
                if is_digit(dateNum) and is_digit(monthNum):
                    text=text[:index] + text[index+len(month)-1:]
                    temp = list(text)
                    temp[index]='/'
                    text = "".join(temp)
        except Exception as e:
            if str(e) in 'substring not found':
                pass
            else:
                raise e
        try:
            if (text.index(year,index)==index):
                monthNum = text[index -1]
                yearNum = text[index + len(year)]
                if is_digit(monthNum) and is_digit(yearNum):
                    text=text[:index] + text[index+len(year)-1:]
                    temp = list(text)
                    temp[index]='/'
                    text = "".join(temp)
        except Exception as e:
            if str(e) in 'substring not found':
                pass
            else:
                raise e
    return text


In [2]:
def Transcribe_Short_Audio(Audio_wav,config_wav_enhanced):
    client = speech.SpeechClient()
    with io.open(Audio_wav, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    # print(type(audio))
    
    response = client.recognize(config=config_wav_enhanced, audio=audio)
    text = []
    for i, result in enumerate(response.results):
        alternative = result.alternatives[0]
        alter=ConvertDate(alternative.transcript+ '\n')
        text.append(alter)
    return text

In [3]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../Speech2Text.json'

In [4]:
os.getcwd()

'c:\\Users\\quang\\Desktop\\chominh\\s2t_GCP_Final\\Funtion_Task'

In [6]:
path = '../Input/1'
path = os.path.abspath(path)

In [7]:
def get_frame_rate(audio_file):
    rate,channel = frame_rate_channel(audio_file)
    config = Config_GGC(sample_rate_hertz=rate,
    audio_channel_count=channel)
    return config

In [9]:
path

'c:\\Users\\quang\\Desktop\\chominh\\s2t_GCP_Final\\Input\\1'

In [32]:
from concurrent.futures import ThreadPoolExecutor
from threading import current_thread
import time

threadnum=4
def func_thread(path):
    config = get_frame_rate(path)
    now = time.time()
    text = Transcribe_Short_Audio(path, config)
    thread = current_thread()
    end_time = time.time() - now
    x = [thread.name, end_time]
    return x

def text_multithread(audio_file):
    now = time.time()
    with ThreadPoolExecutor(threadnum) as executor:
        futures = executor.map(func_thread, path)
        Qlist = [x for x in futures]
    end_time = time.time() - now
    print(f"kết quả threads = {threadnum}")
    full_time = 0
    for x in Qlist:
        print(x[0] + ' || ' + x[1])
        full_time += x[1]
    print(f'Số lần đưa lên GGC : {len(Qlist)}, multi time: {end_time}, full time: {full_time}')

In [30]:
count = 1
path = path + '\\' + 'file_' + str(count) + '.wav'
path

'c:\\Users\\quang\\Desktop\\chominh\\s2t_GCP_Final\\Input\\1\\file_1.wav'

In [33]:
text_multithread(path)

FileNotFoundError: [Errno 2] No such file or directory: 'c'